In [1]:
!pip install scholarly

In [4]:
!pip3 install -U scholarly
!sudo apt-get install tor

In [5]:
!git clone https://github.com/yourusername/AutoSurveyGPT.git


'Can Generative AI Improve Social Science Research?[1]\n\nChris Bail\nDuke University\nwww.chrisbail.net\n\nAbstract. Artificial intelligence that can produce realistic text, images, and\nother human-like outputs is currently transforming many different industries.\nYet it is not yet known how such tools might transform social science research. In\nthe first section of this article, I assess the potential of Generative AI to improve\nonline experiments, agent-based models, and automated content analyses. I also\ndiscuss whether these tools may help social scientists perform literature reviews,\nidentify novel research questions, and develop hypotheses to explain them. Next,\nI evaluate whether Generative AI can help social scientists with more mundane\ntasks such as acquiring advanced programming skills or writing more effective\nprose. In the second section of this article I discuss the limitations of Generative\nAI as well as how these tools might be employed by researchers in an eth

In [12]:
import random
from scholarly import scholarly,ProxyGenerator
from pprint import pprint

pg = ProxyGenerator()
pg.FreeProxies()
scholarly.use_proxy(pg)
scholarly.search_pubs('Large Batch Optimization for Deep Learning: Training BERT in 76 minutes')



In [13]:
from scholarly import scholarly

# Retrieve the author's data, fill-in, and print
# Get an iterator for the author results
search_query = scholarly.search_author('( "social simulation" OR "social modelling" ) AND ( "Large Language Models" OR "LLMs" ) AND "Computational Social Science"')

True

In [2]:
search_query = scholarly.search_pubs('simulation')
scholarly.pprint(next(search_query))

MaxTriesExceededException: Cannot Fetch from Google Scholar.

In [20]:
search_query = scholarly.search_keyword('simulation')
scholarly.pprint(next(search_query))

b"{'affiliation': 'Chubu University',\n 'citedby': 240049,\n 'email_domain': '@slac.stanford.edu',\n 'filled': False,\n 'interests': ['Simulation', 'High Energy Physics', 'Cosmic Ray Physics'],\n 'name': 'Tatsumi Koi',\n 'scholar_id': 'CsO0RukAAAAJ',\n 'source': 'SEARCH_AUTHOR_SNIPPETS',\n 'url_picture': 'https://scholar.google.com/citations?view_op=medium_photo&user=CsO0RukAAAAJ'}"


In [11]:
scholarly.pprint(next(search_query))


b"{'affiliation': 'Danish Broadcasting Corporation, PhD Niels Bohr Institute',\n 'citedby': 122102,\n 'email_domain': '@nbi.dk',\n 'filled': False,\n 'interests': ['Particle physics',\n               'multivariate methods',\n               'simulation',\n               'outreach',\n               'innovation'],\n 'name': 'Ask Emil Loevschall-Jensen',\n 'scholar_id': 'PzROYKsAAAAJ',\n 'source': 'SEARCH_AUTHOR_SNIPPETS',\n 'url_picture': 'https://scholar.google.com/citations?view_op=medium_photo&user=PzROYKsAAAAJ'}"


In [1]:
import csv

from selenium import webdriver
from selenium.webdriver.chrome.options import Options

from bs4 import BeautifulSoup
import pandas as pd
import requests
import re
import logging
import configparser
import time


In [3]:

# Basic configs:
logging.basicConfig(filename='log.log', level=logging.INFO, format='%(asctime)s %(message)s')
config = configparser.ConfigParser()
config.read('config.ini')

# Load PICOC terms
picoc = {
    'population': re.compile(config['picoc']['population']) if config['picoc']['population'] != "" else None,
    'intervention': re.compile(config['picoc']['intervention']) if config['picoc']['intervention'] != "" else None,
    'comparison': re.compile(config['picoc']['comparison']) if config['picoc']['comparison'] != "" else None,
    'outcome': re.compile(config['picoc']['outcome']) if config['picoc']['outcome'] != "" else None,
    'context': re.compile(config['picoc']['context']) if config['picoc']['context'] != "" else None
}

# Create a new Chorme session

options = None
if config['default']['binary_location']:
    options = Options()
    options.binary_location = config['default']['binary_location']

In [9]:
driver = webdriver.Chrome(r"C:\Users\karet\Downloads\chromedriver_win32\chromedriver", options=options)
url = "https://scholar.google.com/"
driver.get(url)

# Setting Google Scholar
driver.maximize_window()
time.sleep(2)
driver.find_element_by_id("gs_hdr_mnu").click()
time.sleep(2)

driver.find_element_by_class_name("gs_btnP").click()
time.sleep(2)

driver.find_element_by_id("gs_num-b").click()
time.sleep(2)

driver.find_element_by_css_selector('a[data-v="20"').click()
time.sleep(2)

driver.find_element_by_id("gs_settings_import_some").click()
time.sleep(2)

driver.find_element_by_name("save").click()



In [12]:
query = '( "social simulation" OR "social modelling" ) AND ( "Large Language Models" OR "LLMs" ) AND "Computational Social Science"'
year= 2023


driver.get(url + "scholar?hl=en&q={0}&as_sdt=1&as_vis=1&as_ylo={1}&as_yhi={1}".format(query, year))


In [23]:
def parser(soup, page, year):
    papers = []
    html = soup.findAll('div', {'class': 'gs_r gs_or gs_scl'})
    for result in html:
        paper = {'Link': result.find('h3', {'class': "gs_rt"}).find('a')['href'], 'Additional link': '', 'Title': '',
                 'Authors': '', 'Abstract': '', 'Cited by': '', 'Cited list': '', 'Related list': '', 'Bibtex': '',
                 'Year': year, 'Google page': page}

        # If it does not pass at Title-Abstract-Keyword filter exclude this paper and continue
        if not filterTitleAbsKey(paper['Link']):
            continue

        try:
            paper["Additional link"] = result.find('div', {'class': "gs_or_ggsm"}).find('a')['href']
        except:
            paper["Additional link"] = ''
            print("NOTHING WAS FOUND")

        paper['Title'] = result.find('h3', {'class': "gs_rt"}).text
        paper['Authors'] = ";".join(
            ["%s:%s" % (a.text, a['href']) for a in result.find('div', {'class': "gs_a"}).findAll('a')])

        try:
            paper['Abstract'] = result.find('div', {'class': "gs_rs"}).text
        except:
            print("NOTHING WAS FOUND")
            paper['Abstract'] = ''

        for a in result.findAll('div', {'class': "gs_fl"})[-1].findAll('a'):
            if a.text != '':
                if a.text.startswith('Cited'):
                    paper['Cited by'] = a.text.rstrip().split()[-1]
                    paper['Cited list'] = url + a['href']
                if a.text.startswith('Related'):
                    paper['Related list'] = url + a['href']
                if a.text.startswith('Import'):
                    paper['Bibtex'] = requests.get(a['href']).text
                    
        papers.append(paper)
        # Wait 20 seconds until the next request to google
        time.sleep(20)

    return papers, len(html)

In [24]:
def filterTitleAbsKey(site):
    try:
        page = requests.get(site,timeout=600)
        text = BeautifulSoup(page.text, 'lxml').get_text()
        text = str.lower(text)
        for terms in filter(None, picoc.values()):
            if not terms.search(text):
                logging.info("%s not passed on title-abs-key filter", site)
                return False
        logging.info("%s passed on title-abs-key filter", site)
        return True
    except requests.exceptions.Timeout:
        logging.info("[TIMEOUT] Timeout on %s and not passed on title-abs-key filter. Skipping website", site)
    except:
        logging.info("[ERROR] on %s and not passed on title-abs-key filter", site)
    return False

In [25]:
art, t = parser(BeautifulSoup(driver.page_source, 'lxml'), 1, 2023)

In [27]:
html = BeautifulSoup(driver.page_source, 'lxml').findAll('div', {'class': 'gs_r gs_or gs_scl'})

In [30]:
for result in html:
    paper = {'Link': result.find('h3', {'class': "gs_rt"}).find('a')['href'], 'Additional link': '', 'Title': '',
             'Authors': '', 'Abstract': '', 'Cited by': '', 'Cited list': '', 'Related list': '', 'Bibtex': '',
             'Year': year, 'Google page': 1}
    print(paper)

{'Link': 'https://osf.io/rwtzs/download', 'Additional link': '', 'Title': '', 'Authors': '', 'Abstract': '', 'Cited by': '', 'Cited list': '', 'Related list': '', 'Bibtex': '', 'Year': 2023, 'Google page': 1}
{'Link': 'https://arxiv.org/abs/2307.14984', 'Additional link': '', 'Title': '', 'Authors': '', 'Abstract': '', 'Cited by': '', 'Cited list': '', 'Related list': '', 'Bibtex': '', 'Year': 2023, 'Google page': 1}
{'Link': 'https://arxiv.org/abs/2308.11432', 'Additional link': '', 'Title': '', 'Authors': '', 'Abstract': '', 'Cited by': '', 'Cited list': '', 'Related list': '', 'Bibtex': '', 'Year': 2023, 'Google page': 1}
{'Link': 'https://search.proquest.com/openview/0bbaae522618c888853a4c6dbfa6a58e/1?pq-origsite=gscholar&cbl=18750&diss=y', 'Additional link': '', 'Title': '', 'Authors': '', 'Abstract': '', 'Cited by': '', 'Cited list': '', 'Related list': '', 'Bibtex': '', 'Year': 2023, 'Google page': 1}
{'Link': 'https://research-information.bris.ac.uk/files/364550531/FINAL_DRAFT_